In [ ]:




#导入torch，用于构建神经网络
import scipy.io as sio
import torch
from torch import nn
import math


import numpy as np
import pandas as pd

#记录时间
from datetime import datetime

import time


import matplotlib.pyplot as plt

from scipy.interpolate import griddata



#用于重复独立实验的随机种子
import random


#获取当前文件的名称

import os



In [ ]:


#设置随机种子
def setup_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cuda.matmul.allow_tf32 = False# 尝试禁用这两个数据类型
        torch.backends.cudnn.allow_tf32 = False
        #torch.backends.cudnn.deterministic = True
        #torch.backends.cudnn.benchmark = False

        



        
def sample(min,max, N):

    x = min + (max - min) * torch.rand((N, 1), dtype = datadtype).to(device).requires_grad_(True)
   
    return x



In [ ]:
class Net(nn.Module):
    def __init__(self, zysdtype,layers):
        super().__init__()#将父类的方法继承过来

        self.activation = nn.Tanh#选择tanh函数作为激活函数
        #nn.tanh是一个类，nn.Tanh（）是一个函数

        self.dtype = zysdtype

       
        self.depth = len(layers) - 1
        

        self.layers = nn.Sequential()

        for i in range(self.depth -1):
            self.layers.add_module('layer_%d' %i,
                                   nn.Linear(layers[i], layers[i+1],dtype = self.dtype))
            
            self.layers.add_module('activation_%d'%i,self.activation())

        #最后一层不通过激活函数
        self.layers.add_module("layer_%d" %(self.depth -1),nn.Linear(layers[-2], layers[-1],dtype = self.dtype))



      

    def forward(self, t, x, y):
        # #对输入进行归一化
        # t = 2.0*(t - t_lb)/(t_ub - t_lb) - 1.0
        # x = 2.0*(x - x_min)/(x_max - x_min) - 1.0

        H = torch.cat([t, x, y], 1)
        
        return self.layers(H)


In [ ]:


# %%

class HCPINN(nn.Module):
    def __init__(self,mydtype, layers,  t_min,t_max):
        
        super(HCPINN, self).__init__()
        self.dtype = mydtype
        self.layers = layers

        self.t_min = t_min
        self.t_max = t_max



   
        #采点
        self.N_i = 1200
        self.N_b = int(1200 * (self.t_max - self.t_min))
        self.N_r = int(500 * (self.t_max - self.t_min))
        self.eta = 0.6 #定义重采样率
        self.N_n = int(self.N_r * self.eta)
        
        self.t_ics_batch =  sample(self.t_min,self.t_min, self.N_i)
        self.x_ics_batch =  sample(x_min,x_max, self.N_i)
        self.y_ics_batch =  sample(y_min,y_max, self.N_i)

       

        
        self.t_bc1_batch =  sample(self.t_min,self.t_max, self.N_b)
        self.x_bc1_batch =  sample(x_min,x_min, self.N_b)
        self.y_bc1_batch =  sample(y_min,y_max, self.N_b)
     
        
        self.x_bc2_batch =  sample(x_max,x_max, self.N_b)
        self.y_bc2_batch =  sample(y_min,y_max, self.N_b)
       

        self.x_bc3_batch =  sample(x_min,x_max, self.N_b)
        self.y_bc3_batch =  sample(y_min,y_min, self.N_b)
     
        self.x_bc4_batch =  sample(x_min,x_max, self.N_b)
        self.y_bc4_batch =  sample(y_max,y_max, self.N_b)
    
   


        #Fetch residual mini-bathes
        
        self.t_res_batch =  sample(self.t_min,self.t_max, self.N_r)
        self.x_res_batch =  sample(x_min,x_max, self.N_r)
        self.y_res_batch =  sample(y_min,y_max, self.N_r)
   
        #V的定解条件
        self.V_ics_batch = exact_V(self.t_ics_batch, self.x_ics_batch, self.y_ics_batch)
        self.V_bc1_batch = exact_V(self.t_bc1_batch, self.x_bc1_batch, self.y_bc1_batch)
        self.V_bc2_batch = exact_V(self.t_bc1_batch, self.x_bc2_batch, self.y_bc2_batch)
        self.V_bc3_batch = exact_V(self.t_bc1_batch, self.x_bc3_batch, self.y_bc3_batch)
        self.V_bc4_batch = exact_V(self.t_bc1_batch, self.x_bc4_batch, self.y_bc4_batch)

        #B的定解条件
        self.B_ics_batch = exact_B(self.t_ics_batch, self.x_ics_batch, self.y_ics_batch)
        self.B_bc1_batch = exact_B(self.t_bc1_batch, self.x_bc1_batch, self.y_bc1_batch)
        self.B_bc2_batch = exact_B(self.t_bc1_batch, self.x_bc2_batch, self.y_bc2_batch)
        self.B_bc3_batch = exact_B(self.t_bc1_batch, self.x_bc3_batch, self.y_bc3_batch)
        self.B_bc4_batch = exact_B(self.t_bc1_batch, self.x_bc4_batch, self.y_bc4_batch)
    
    
        #定义优化器
        self.net = Net(self.dtype, self.layers).to(device)
        self.iter = 0
        self.adam = torch.optim.Adam(self.parameters(), lr = 1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.adam, step_size = 1000, gamma= 0.9)
        self.optimizer = torch.optim.LBFGS(
            self.parameters(),
            lr = 1.0,
            max_iter = 10000,
            max_eval = 50000,
            history_size = 200,
            tolerance_grad = 1e-12,
            tolerance_change = 1.0 * np.finfo(float).eps,
            line_search_fn = 'strong_wolfe'
        )  
    
    def net_u(self, t,x,y):
        result = self.net(t,x,y)
        return result[:,0:1], result[:, 1:2]# V,B
    
    def net_r(self, t, x,y):

        V,B = self.net_u(t,x,y)
       

        V_t = torch.autograd.grad(V, t,
                                  grad_outputs=torch.ones_like(V),
                                  create_graph= True,
                                  retain_graph=True)[0]
                
        V_x = torch.autograd.grad(V, x,
                                  grad_outputs=torch.ones_like(V),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        V_xx = torch.autograd.grad(V_x, x,
                                  grad_outputs=torch.ones_like(V_x),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        V_y = torch.autograd.grad(V, y,
                                  grad_outputs=torch.ones_like(V),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        V_yy = torch.autograd.grad(V_y, y,
                                  grad_outputs=torch.ones_like(V_y),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_x = torch.autograd.grad(B, x,
                                  grad_outputs=torch.ones_like(B),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_xx = torch.autograd.grad(B_x, x,
                                  grad_outputs=torch.ones_like(B_x),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_y = torch.autograd.grad(B, y,
                                  grad_outputs=torch.ones_like(B),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_yy = torch.autograd.grad(B_y, y,
                                  grad_outputs=torch.ones_like(B_y),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_t = torch.autograd.grad(B, t,
                                  grad_outputs=torch.ones_like(B),
                                  create_graph= True,
                                  retain_graph=True)[0]
        

        residual_r1 = V_t - vare * (V_xx + V_yy) + a1 * B_x + a2 * B_y
        residual_r2 = B_t - vare * (B_xx + B_yy) + a1 * V_x + a2 * V_y
                                   
        return residual_r1, residual_r2
    
   


    def closure(self):
        self.adam.zero_grad()
        self.optimizer.zero_grad()



        #Evaluate predictions
        
      
        self.V_ics_pred,self.B_ics_pred = self.net_u(self.t_ics_batch, self.x_ics_batch, self.y_ics_batch)

        self.V_bc1_pred,self.B_bc1_pred = self.net_u(self.t_bc1_batch, self.x_bc1_batch, self.y_bc1_batch)
        self.V_bc2_pred,self.B_bc2_pred = self.net_u(self.t_bc1_batch, self.x_bc2_batch, self.y_bc2_batch)
        self.V_bc3_pred,self.B_bc3_pred = self.net_u(self.t_bc1_batch, self.x_bc3_batch, self.y_bc3_batch)
        self.V_bc4_pred,self.B_bc4_pred = self.net_u(self.t_bc1_batch, self.x_bc4_batch, self.y_bc4_batch)
       
     
        self.r1_pred,self.r2_pred = self.net_r(self.t_res_batch, self.x_res_batch, self.y_res_batch)

        #Boundary loss and Initial loss
        self.loss_ic = torch.mean((self.V_ics_pred- self.V_ics_batch)**2 ) + torch.mean((self.B_ics_pred- self.B_ics_batch)**2 ) 
        

        self.loss_ics = self.loss_ic 


        self.loss_bc1 = torch.mean((self.V_bc1_pred - self.V_bc1_batch)**2 ) + torch.mean((self.B_bc1_pred - self.B_bc1_batch)**2 )
        self.loss_bc2 = torch.mean((self.V_bc2_pred - self.V_bc2_batch)**2 ) + torch.mean((self.B_bc2_pred - self.B_bc2_batch)**2 )
        self.loss_bc3 = torch.mean((self.V_bc3_pred - self.V_bc3_batch)**2 ) + torch.mean((self.B_bc3_pred - self.B_bc3_batch)**2 )
        self.loss_bc4 = torch.mean((self.V_bc4_pred - self.V_bc4_batch)**2 ) + torch.mean((self.B_bc4_pred - self.B_bc4_batch)**2 )             

        self.loss_bcs =  self.loss_bc1 + self.loss_bc2 + self.loss_bc3 + self.loss_bc4
        

        #Residual loss
        self.loss_res = torch.mean((self.r1_pred)**2 ) + torch.mean((self.r2_pred)**2 )

        #Total loss
        self.loss =  self.loss_ics + self.loss_bcs + self.loss_res 

        self.loss.backward(retain_graph=True)
        

        self.iter += 1
        

        if self.iter % 5000 == 0:
            print(f'\r{self.iter},loss:{self.loss.item():.5e},i:{self.loss_ics.item():.2e},f:{self.loss_res.item():.2e},b:{self.loss_bcs.item():.2e}')
            if self.iter < nIter:
            
                x_new_train = sample(x_min,x_max, self.N_n)

                # 生成随机索引
                shuffle_indices = torch.randperm(self.N_r, device = device)

                # 打乱张量顺序
                self.x_res_batch = self.x_res_batch[shuffle_indices]

                self.x_res_batch[0:self.N_n] = x_new_train


        return self.loss
    
    def predict_u(self,X):
        t = X[:,0:1]
        x = X[:,1:2]
        y = X[:,2:3]

        #模型进入评估状态
        self.eval()
        with torch.no_grad():
            V,B = self.net_u(t, x, y)
 
        return V.detach(),B.detach()
    




In [ ]:


# %%

class HCPINN_next(nn.Module):
    def __init__(self,  mydtype, layers,  t_min,  t_max, premodel):
        
        super(HCPINN_next, self).__init__()

        self.premodel = premodel
        self.dtype = mydtype
        self.layers = layers

        self.t_min = t_min
        self.t_max = t_max

        #停止第一个网络参数的训练
        for param in self.premodel.net.parameters():
            param.requires_grad = False


   
        #采点
        self.N_i = 2
        self.N_b = int(1000 * (self.t_max - self.t_min))
        self.N_r = int(500 * (self.t_max - self.t_min))
        self.eta = 0.6 #定义重采样率
        self.N_n = int(self.N_r * self.eta)
        
        self.t_ics_batch =  sample(self.t_min,self.t_min, self.N_i)
        self.x_ics_batch =  sample(x_min,x_max, self.N_i)
        self.y_ics_batch =  sample(y_min,y_max, self.N_i)


        
        self.t_bc1_batch =  sample(self.t_min,self.t_max, self.N_b)
        self.x_bc1_batch =  sample(x_min,x_min, self.N_b)
        self.y_bc1_batch =  sample(y_min,y_max, self.N_b)
   

        self.x_bc2_batch =  sample(x_max,x_max, self.N_b)
        self.y_bc2_batch =  sample(y_min,y_max, self.N_b)
     
        self.x_bc3_batch =  sample(x_min,x_max, self.N_b)
        self.y_bc3_batch =  sample(y_min,y_min, self.N_b)
       
        self.x_bc4_batch =  sample(x_min,x_max, self.N_b)
        self.y_bc4_batch =  sample(y_max,y_max, self.N_b)
       
        
        #Fetch residual mini-bathes
        
        self.t_res_batch =  sample(self.t_min,self.t_max, self.N_r)
        self.x_res_batch =  sample(x_min,x_max, self.N_r)
        self.y_res_batch =  sample(y_min,y_max, self.N_r)
       

        #V的定解条件
        self.V_ics_batch = exact_V(self.t_ics_batch, self.x_ics_batch, self.y_ics_batch)
        self.V_bc1_batch = exact_V(self.t_bc1_batch, self.x_bc1_batch, self.y_bc1_batch)
        self.V_bc2_batch = exact_V(self.t_bc1_batch, self.x_bc2_batch, self.y_bc2_batch)
        self.V_bc3_batch = exact_V(self.t_bc1_batch, self.x_bc3_batch, self.y_bc3_batch)
        self.V_bc4_batch = exact_V(self.t_bc1_batch, self.x_bc4_batch, self.y_bc4_batch)

        #B的定解条件
        self.B_ics_batch = exact_B(self.t_ics_batch, self.x_ics_batch, self.y_ics_batch)
        self.B_bc1_batch = exact_B(self.t_bc1_batch, self.x_bc1_batch, self.y_bc1_batch)
        self.B_bc2_batch = exact_B(self.t_bc1_batch, self.x_bc2_batch, self.y_bc2_batch)
        self.B_bc3_batch = exact_B(self.t_bc1_batch, self.x_bc3_batch, self.y_bc3_batch)
        self.B_bc4_batch = exact_B(self.t_bc1_batch, self.x_bc4_batch, self.y_bc4_batch)
    
    

                #将分段函数加入训练
        self.breakpoint_log = []
        

        self.breakpoint_lb = self.t_min.item()
        self.breakpoint_ub = self.t_max.item()
    
        self.breakpoint = nn.Parameter(self.t_min+(self.t_max-self.t_min)*0.5)
        print("p的初始化：%.3e "%self.breakpoint.item())


        #定义优化器
        self.net = Net(self.dtype, self.layers).to(device)
        self.iter = 0
        self.adam = torch.optim.Adam(self.parameters(), lr = 1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.adam, step_size = 1000, gamma= 0.9)
        self.optimizer = torch.optim.LBFGS(
            self.parameters(),
            lr = 1.0,
            max_iter = 10000,
            max_eval = 50000,
            history_size = 200,
            tolerance_grad = 1e-12,
            tolerance_change = 1.0 * np.finfo(float).eps,
            line_search_fn = 'strong_wolfe'
        )

    

    def control_fun(self,t):
        ab = self.breakpoint
        # 条件1: t <= self.t_min
        condition2 = t <= self.t_min
        # 条件3: t <= ab
        condition3 = t <= ab
        # 条件4: ab <=self.t_end
        condition4 = ab <= self.t_max

        # 计算 B 和 BB
        A = (t - self.t_min) / (self.t_max - self.t_min)
        AA = A**2
        B = (t - self.t_min) / (ab - self.t_min)
        BB = B**2

        # 定义分段函数
        result = torch.where(condition2, torch.zeros_like(t),
                        torch.where(condition4, 
                                    torch.where(condition3, -2 * B*BB + 3 * BB, torch.zeros_like(t) + 1),
                                    -2 * A*AA + 3 * AA))
        return result
  
    
    def net_u(self, t,x,y):
        weight1 = self.control_fun(t)
        result = ( 1-weight1)*self.premodel.net(t,x,y) +weight1 * self.net(t,x,y)
        return result[:,0:1], result[:, 1:2]# V,B
    
    def net_r(self, t, x,y):

        V,B = self.net_u(t,x,y)
       

        V_t = torch.autograd.grad(V, t,
                                  grad_outputs=torch.ones_like(V),
                                  create_graph= True,
                                  retain_graph=True)[0]
                
        V_x = torch.autograd.grad(V, x,
                                  grad_outputs=torch.ones_like(V),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        V_xx = torch.autograd.grad(V_x, x,
                                  grad_outputs=torch.ones_like(V_x),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        V_y = torch.autograd.grad(V, y,
                                  grad_outputs=torch.ones_like(V),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        V_yy = torch.autograd.grad(V_y, y,
                                  grad_outputs=torch.ones_like(V_y),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_x = torch.autograd.grad(B, x,
                                  grad_outputs=torch.ones_like(B),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_xx = torch.autograd.grad(B_x, x,
                                  grad_outputs=torch.ones_like(B_x),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_y = torch.autograd.grad(B, y,
                                  grad_outputs=torch.ones_like(B),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_yy = torch.autograd.grad(B_y, y,
                                  grad_outputs=torch.ones_like(B_y),
                                  create_graph= True,
                                  retain_graph=True)[0]
        
        B_t = torch.autograd.grad(B, t,
                                  grad_outputs=torch.ones_like(B),
                                  create_graph= True,
                                  retain_graph=True)[0]
        

        residual_r1 = V_t - vare * (V_xx + V_yy) + a1 * B_x + a2 * B_y
        residual_r2 = B_t - vare * (B_xx + B_yy) + a1 * V_x + a2 * V_y
                                   
        return residual_r1, residual_r2
    
   
    

    
    def closure(self):
        self.adam.zero_grad()
        self.optimizer.zero_grad()



        #Evaluate predictions
    
    
        self.breakpoint_log.append([self.breakpoint.item()])
        
      
        self.V_ics_pred,self.B_ics_pred = self.net_u(self.t_ics_batch, self.x_ics_batch, self.y_ics_batch)

        self.V_bc1_pred,self.B_bc1_pred = self.net_u(self.t_bc1_batch, self.x_bc1_batch, self.y_bc1_batch)
        self.V_bc2_pred,self.B_bc2_pred = self.net_u(self.t_bc1_batch, self.x_bc2_batch, self.y_bc2_batch)
        self.V_bc3_pred,self.B_bc3_pred = self.net_u(self.t_bc1_batch, self.x_bc3_batch, self.y_bc3_batch)
        self.V_bc4_pred,self.B_bc4_pred = self.net_u(self.t_bc1_batch, self.x_bc4_batch, self.y_bc4_batch)
     
     
        self.r1_pred,self.r2_pred = self.net_r(self.t_res_batch, self.x_res_batch, self.y_res_batch)

        #Boundary loss and Initial loss
        self.loss_ic = torch.mean((self.V_ics_pred- self.V_ics_batch)**2 ) + torch.mean((self.B_ics_pred- self.B_ics_batch)**2 ) 
        

        self.loss_ics = self.loss_ic 


        self.loss_bc1 = torch.mean((self.V_bc1_pred - self.V_bc1_batch)**2 ) + torch.mean((self.B_bc1_pred - self.B_bc1_batch)**2 )
        self.loss_bc2 = torch.mean((self.V_bc2_pred - self.V_bc2_batch)**2 ) + torch.mean((self.B_bc2_pred - self.B_bc2_batch)**2 )
        self.loss_bc3 = torch.mean((self.V_bc3_pred - self.V_bc3_batch)**2 ) + torch.mean((self.B_bc3_pred - self.B_bc3_batch)**2 )
        self.loss_bc4 = torch.mean((self.V_bc4_pred - self.V_bc4_batch)**2 ) + torch.mean((self.B_bc4_pred - self.B_bc4_batch)**2 )             

        self.loss_bcs =  self.loss_bc1 + self.loss_bc2 + self.loss_bc3 + self.loss_bc4
        
        

        #Residual loss
        self.loss_res = torch.mean((self.r1_pred)**2 ) + torch.mean((self.r2_pred)**2 )


        #Total loss
        self.loss =   self.loss_bcs + self.loss_res 

        self.loss.backward(retain_graph=True)
        

        self.iter += 1


        if self.iter % 5000 == 0 :
            print(f'\r{self.iter},p:{self.breakpoint.item():.2e},loss:{self.loss.item():.5e},i:{self.loss_ics.item():.2e},f:{self.loss_res.item():.2e},b:{self.loss_bcs.item():.2e}')
            if self.iter < nIter:
            
                x_new_train = sample(x_min,x_max, self.N_n)

                # 生成随机索引
                shuffle_indices = torch.randperm(self.N_r, device = device)

                # 打乱张量顺序
                self.x_res_batch = self.x_res_batch[shuffle_indices]

                self.x_res_batch[0:self.N_n] = x_new_train

        return self.loss
    
    def predict_u(self,X):
        t = X[:,0:1]
        x = X[:,1:2]
        y = X[:,2:3]
    
        #模型进入评估状态
        self.eval()
        with torch.no_grad():
            V,B = self.net_u(t, x, y)
 
        return V.detach(),B.detach()
    




In [ ]:

vare = 0.1
gama = math.pi / 4
a1, a2 = -math.sin(gama), -math.cos(gama)

#定义真实解
def exact_V(t, x, y):
    result = torch.exp( -2 * vare * torch.pi ** 2 * t) * torch.cos(torch.pi * (a1 + a2) * t) * torch.sin(torch.pi * (x + y))

    return result

def Exact_V(X):
    '''
    
    :param x: x = (t,x)
    '''
    t = X[:, 0:1]
    x = X[:, 1:2]
    y = X[:, 2:3]

    result = torch.exp( -2 * vare * torch.pi ** 2 * t) * torch.cos(torch.pi * (a1 + a2) * t) * torch.sin(torch.pi * (x + y))
    
    return result


def exact_B(t, x, y):
    result =- torch.exp( -2 * vare * torch.pi ** 2 * t) * torch.sin(torch.pi * (a1 + a2) * t) * torch.cos(torch.pi * (x + y))

    return result


def Exact_B(X):
    '''
    
    :param x: x = (t,x)
    '''
    t = X[:, 0:1]
    x = X[:, 1:2]
    y = X[:, 2:3]

    result = - torch.exp( -2 * vare * torch.pi ** 2 * t) * torch.sin(torch.pi * (a1 + a2) * t) * torch.cos(torch.pi * (x + y))
    
    return result

In [ ]:

#画出区间的图像以及求误差
def plot_error_block(model,t_min,t_max):
   
    n_min = int(t_min * num_points)
    n_max = int(t_max * num_points)
    #n_test = n_max - n_min

    t_block = t_total[n_min:n_max +1]
    T_block, X_block, Y_block= torch.meshgrid(t_block, x_total, y_total,indexing='ij')

    #组装
 
    X_star_block = torch.cat((T_block.flatten()[:, None], X_block.flatten()[:,None], Y_block.flatten()[:,None]), dim=1)

    #引入真实的信息
    V0_star = Exact_V(X_star_block)
    V_star = V0_star.flatten()[:,None]
    B0_star = Exact_B(X_star_block)
    B_star = B0_star.flatten()[:,None]

    #网络预测的结果
    V_pred_block,B_pred_block = model.predict_u(X_star_block)

    #计算误差
    L2_V =  torch.norm(V_star - V_pred_block, p=2) /torch.norm(V_star, p=2)
    L1_V = torch.norm(V_star - V_pred_block, p=1) / len(X_star_block)
    Linf_V = torch.norm(V_star - V_pred_block, p=float('inf'))

    L2_B =  torch.norm(B_star - B_pred_block, p=2) /torch.norm(B_star, p=2)
    L1_B = torch.norm(B_star - B_pred_block, p=1) / len(X_star_block)
    Linf_B = torch.norm(B_star - B_pred_block, p=float('inf'))
    
    print("\n查看区间：[%.2f,%.2f]"%(T_block.min().item(), T_block.max().item()))
    print("\nV L2,L1,Linf: (%.2e, %.2e, %.2e)}"%(L2_V,L1_V, Linf_V))
    print("\nB L2,L1,Linf: (%.2e, %.2e, %.2e)}"%(L2_B,L1_B, Linf_B))
   


    return V_pred_block,B_pred_block, L2_V.item(), L1_V.item(), Linf_V.item(), L2_B.item(), L1_B.item(), Linf_B.item()




In [ ]:
def plot_func(t, func1,p_log):
    fig = plt.figure(figsize=(15, 3),dpi=200)
    plt.subplot(1, 2, 1)
    plt.plot(t.cpu().detach().numpy(), func1(t).cpu().detach().numpy())
    #plt.title(r"$\lamda(t)$")


    plt.subplot(1, 2, 2)
    plt.plot(range(0, len(p_log[::10])), p_log[::10])
    #plt.title(r"$\eta(t)$")
    plt.show()


In [ ]:



def train_block(t_min, t_max, num_block,train_num):
    train_time = 0
    models = []
    error_total = []
   
    t_block = torch.linspace(t_min, t_max, num_block + 1).to(device)

    train_time_total = []
    for i in range(num_block):
        
        print('训练区间：[%.2f,%.2f]'%(t_block[i], t_block[i+1]))

        
        if i == 0:
            
            model = HCPINN(datadtype,layers, t_block[i],t_block[i+1])

                #进行训练
            start_time = time.time()
            print('\n\t====Begin Adam====')
            for adam_iter in range(nIter):
                    model.closure()
                    model.adam.step()
                    model.scheduler.step()
            print('\n\t====Begin LBFSG====')
            model.optimizer.step(model.closure)
            block_time =  (time.time() - start_time)/60

       


        else:

            model = HCPINN_next(datadtype, layers,t_block[i], t_block[i+1],   models[-1])



            #进行训练
            start_time = time.time()
            print('\n\t====Begin Adam====')
            for adam_iter in range(nIter):
                    model.closure()
                    model.adam.step()
                    model.scheduler.step()
            print('\n\t====Begin LBFSG====')
            model.optimizer.step(model.closure)
            block_time =  (time.time() - start_time)/60

                #保存w的变化
            df_w = pd.DataFrame(model.breakpoint_log)

            # 将DataFrame保存为Excel文件
            df_w.to_excel(folder_path + "/第%d次训练_[%.2f,%.2f]_p_log.xlsx"%(train_num, t_block[i],t_block[i+1]), index=False)

            plot_func(torch.linspace(t_min, t_max + 0.1, 10000).to(device), model.control_fun, model.breakpoint_log)
        
        train_time_total.append(block_time)
        print('区间[%.2f,%.2f]的训练时间：%.2f min(s)'%(t_block[i],t_block[i+1] ,(time.time() - start_time) /60)) 
        print("Final loss: %.5e,Total iteration: %d"%(model.loss.item(),model.iter))

        
        
        #保存网络
        torch.save(model.state_dict(),folder_path + "/第%d次训练_[%.2f,%.2f].pth"%(train_num, t_block[i],t_block[i+1]))

        _,_, *error_block= plot_error_block(model,t_block[i], t_block[i +1])
  

        #记录实验结果
        models.append(model)
        error_total.append(error_block)

    train_time = sum(train_time_total)
    train_time_total.append(train_time)
    print('\n#小区间个数：%d，总训练时长:%.2f  min(s)'%(num_block,train_time))

    return models,error_total, train_time_total




In [ ]:


def u_pred_whole(models, t_min, t_max,num_block,train_times_num):
    
    t_block = torch.linspace(t_min, t_max, num_block + 1).to(device)
    
    n_min = int(t_min * num_points)
    n_max = int(t_max * num_points)

    t_total_now = t_total[n_min : n_max + 1]

    V_combine = []
    V_exact = []
    B_combine = []
    B_exact = []

    for block_index in range(num_block):
        #对每个小区间进行更加详细的计算
        n_min_now = int(t_block[block_index] * num_points)
        n_max_now = int(t_block[block_index +1] * num_points)

        if block_index ==0:            
            tt = t_total_now[n_min_now : n_max_now+1]

        else:
            tt = t_total_now[n_min_now+1 :  n_max_now+1]


        T_pred_now,X_pred_now,Y_pred_now = torch.meshgrid(tt, x_total, y_total, indexing='ij')

        XX_star_pred = torch.cat((T_pred_now.flatten()[:,None], X_pred_now.flatten()[:,None], Y_pred_now.flatten()[:,None]), dim=1)
      

        V_pred,B_pred = models[block_index].predict_u(XX_star_pred)
      
        V_combine.append(V_pred)
        B_combine.append(B_pred)

        V_exact_now = Exact_V(XX_star_pred)
        V_exact.append(V_exact_now)
        B_exact_now = Exact_B(XX_star_pred)
        B_exact.append(B_exact_now)
 
    
    V_com = torch.cat(V_combine, dim=0)
    V_exa = torch.cat(V_exact, dim=0)
    B_com = torch.cat(B_combine, dim=0)
    B_exa = torch.cat(B_exact, dim=0)


    #计算两者的误差
    L2_V = torch.norm(V_com - V_exa, p=2) / torch.norm(V_exa, p=2)    
    L1_V = torch.norm(V_com - V_exa,p=1)/len(V_exa)
    Linf_V = torch.norm(V_com - V_exa, p=float('inf'))

    L2_B = torch.norm(B_com - B_exa, p=2) / torch.norm(B_exa, p=2)  
    L1_B = torch.norm(B_com - B_exa,p=1)/len(B_exa)
    Linf_B = torch.norm(B_com - B_exa, p=float('inf'))

    error_whole = [L2_V.item(), L1_V.item(), Linf_V.item(), L2_B.item(), L1_B.item(), Linf_B.item()]


    print('\n#查看区间[%.2f,%.2f]:'%(t_min, t_max))
    print("\nV L2,L1,Linf: (%.2e, %.2e, %.2e)}"%(L2_V,L1_V, Linf_V))
    print("\nB L2,L1,Linf: (%.2e, %.2e, %.2e)}"%(L2_B,L1_B, Linf_B))


    return V_com,B_com, error_whole





In [ ]:


#定义超参数
datadtype = torch.float32

#CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device( "cpu")

print(f'Using {device} to train...')
print(f'The code is trained at {datetime.now()}')




notebook_name = "result"
print(notebook_name)

import shutil
# 定义文件夹路径
folder_path = notebook_name

# 如果文件夹存在，先删除
if os.path.exists(folder_path):
    
    shutil.rmtree(folder_path)
    print(f"文件夹 '{folder_path}' 已删除。")

# 重新创建文件夹
os.makedirs(folder_path)
print(f"文件夹 '{folder_path}' 已重新创建。")



In [ ]:

# %%

t_min, t_max = 0.0, 60
x_min, x_max = 0, 1
y_min, y_max = 0, 1


#导入数据
num_points = 10

t_total = torch.linspace(0, 60, 601).to(device)

x_total = torch.linspace(x_min, x_max, 31).to(device)

y_total = torch.linspace(y_min, y_max,31).to(device)


#定义独立重复实验次数以及小区间个数


num_block_start = 1
num_block_end = 5
train_times = 5



num_neural = 50

layers = [3, num_neural, num_neural, num_neural,   2]

nIter = 5000



In [ ]:

# %%

names = ['L2_V', 'L1_V', 'Linf_V','L2_B', 'L1_B', 'Linf_B','train_time']


In [ ]:
print('\nvarepslion:%.5f' % vare)

with pd.ExcelWriter(folder_path + '/error.xlsx', engine='openpyxl') as writer:
    for num in range(num_block_start, num_block_end + 1):  # 对每一个训练的区间
        
        Random_seed_log = []
        error_for_train = []  # 用于记录不同区间的总误差
        for train in range(1, train_times + 1):
            
            Random_seed =  random.randint(1, 1000 * train_times)
            Random_seed_log.append(Random_seed)
            setup_seed(Random_seed)
            print('-----**区间个数：%d, 第%d次训练, 随机种子: %d**-----' % (num, train, Random_seed))
            
           
            # 训练模型
            models, error_block, train_time = train_block(t_min, t_max, num, train)
            
            # 全区间预测
            V_pred, B_pred, error_whole = u_pred_whole(models, t_min, t_max, num, train)

            sio.savemat(f'区间个数{num}_第{train}次训练_pred.mat', {
                'V_pred': V_pred.cpu(),
                'B_pred': B_pred.cpu()
            })

    

            # 完整误差记录
            error_block.append(error_whole)
            error_for_train.append(error_block)
            error_for_train.append(train_time)

        error_result = np.column_stack(error_for_train)
        
        # 保存结果
        column_names = [
            f"{Random_seed_log[group_num-1]}_{group_num}_{names[col_num]}"   
            for group_num in range(1, train_times + 1)
            for col_num in range(0, 7)
        ]
        
        index_names = [f'[{t_max * i / num},{t_max * (i + 1) / num}]' for i in range(num + 1)]
        index_names[-1] = f'[{t_min},{t_max}]'
        df1 = pd.DataFrame(error_result, index=index_names, columns=column_names)
        df1.to_excel(writer, sheet_name=f'Sheet_{num}', index=True)

